In [72]:
#!/usr/bin/python3
#BSE_Fourier.ipynb
#Parth Parakh
#purpose: Solving BSM using fourier transform
#---------------------------------------------------------------------
#import modules
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import fft
from scipy import interpolate
from scipy.interpolate import CubicSpline
#----------------------------------------------------------------------
def payoff(x,xi,alpha,K,L,U,C,call):
    #Scale
    S = C*np.exp(x);
    g = np.zeros(len(x))
    #Payoff 
    if call == 1:
        for i in range(0,len(x)):
            # see e.g. Green, Fusai, Abrahams 2010, Eq. (3.24)
            g[i] = np.exp(alpha*x[i])*max(S[i]-K,0)*(S[i]>=L)*(S[i]<=U);
    else  :
        # put
        for i in range(0,len(x)):
            g[i] = np.exp(alpha*x[i])*max(K-S[i],0)*(S[i]>=L)*(S[i]<=U);

    #Analytical Fourier transform of the payoff
    l = np.log(L/C); # lower log barrier
    k = np.log(K/C); # log strike
    u = np.log(U/C); # upper log barrier

    #Integration bounds
    if call == 1:
        # call
        a = max(l,k);
        b = u;
    else:
        #put
        a = min(k,u);
        b = l;

    #Green, Fusai, Abrahams 2010 Eq. (3.26) with extension to put option
    xi2 = alpha+1j*xi;
    G = C*((np.exp(b*(1+xi2))-np.exp(a*(1+xi2)))/(1+xi2) - (np.exp(k+b*xi2)-np.exp(k+a*xi2))/xi2);

    #Eliminable discontinuities for xi = 0, otherwise 0/0 = NaN
    return S,g,G
#-----------------------------------------------------------------------------------
# main caller

    #Defining the parameters
T = 1; # maturity
S0 = 1; # spot price
K = 1.1; #strike price
r = 0.05; #risk-free interest rate
q = 0.02; # dividend rate

sigma = 0.4;

muRN = r-q-0.5*sigma**2;

#Monte Carlo parameters; npaths = nblocks*nsample
nblocks = 20000; #number of blocks
nsample = 10000; #number of samples per block

# Fourier parameters
xwidth = 6; # width of the support in real space
ngrid = 2**8; #number of grid points
alpha = -10; #damping factor for a call

#Analytical Solution
d1 = ((np.log(S0/K)+(r-q+0.5*sigma**2)*T))/(sigma*np.sqrt(T));
# d2 = d1 - sigma*sqrt(T);
d2 = ((np.log(S0/K)+(r-q-0.5*sigma**2)*T))/(sigma*np.sqrt(T));
Vca = (S0*np.exp(-q*T)*norm.cdf(d1)) - (K*np.exp(-r*T)*norm.cdf(d2));
# Put-call parity: V_call + K*exp(-r*T) = V_put - S_0
Vpa = (K*np.exp(-r*T)*norm.cdf(-d2)) - (S0*np.exp(-q*T)*norm.cdf(-d1));

print(Vca)
print(Vpa)

#Fourier transform method

N = ngrid/2;
b = xwidth/2; #upper bound of the support in real space
dx = xwidth/ngrid;
x = dx*np.arange(-N,N-1);
dxi = 2*np.pi/xwidth; #Nyquist relation
xi = dxi*np.arange(-N,N-1);

# Characteristic function at time T
xia = xi+1j*alpha; #call
psi = 1j*muRN*xia-0.5*(sigma*xia)**2; #characteristic exponent
Psic = np.exp(psi*T); #characteristic function
xia = xi-1j*alpha; #put
psi = 1j*muRN*xia-0.5*(sigma*xia)**2; #characteristic exponent
Psip = np.exp(psi*T); #characteristic function

U = S0*np.exp(b);
L = S0*np.exp(-b);

S,gc,Gc = payoff(x,xi,alpha,K,L,U,S0,1); #call
S,gp,Gp = payoff(x,xi,-alpha,K,L,U,S0,0); #put

# Discounted expected payoff computed with the Plancherel theorem
c = np.exp(-r*T)*np.real(fft.fftshift(fft.fft(fft.ifftshift(Gc*np.conjugate(Psic)))))/xwidth; # call
tck = interpolate.splrep(S, c)
VcF = interpolate.splev(S0, tck)
p = np.exp(-r*T)*np.real(fft.fftshift(fft.fft(fft.ifftshift(Gp*np.conjugate(Psip)))))/xwidth; # put
tck = interpolate.splrep(S, p)
VpF = interpolate.splev(S0, tck)

print(VcF,VpF)

0.12965401324786407
0.19580770689189425
0.1121456037249727 0.23369662490673193
